<a href="https://colab.research.google.com/github/vence-andersen/M5-Forecasting-Accuracy/blob/main/EDA_M5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Removables**

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"grahamfernando","key":"cffa546ccec7808cfc8cec9944ed384a"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
! kaggle competitions download -c m5-forecasting-accuracy

100% 15.8M/15.8M [00:00<00:00, 61.9MB/s]

  0% 0.00/163k [00:00<?, ?B/s]
100% 163k/163k [00:00<00:00, 170MB/s]
 32% 5.00M/15.5M [00:00<00:00, 20.5MB/s]
100% 15.5M/15.5M [00:00<00:00, 51.4MB/s]
 49% 7.00M/14.2M [00:00<00:00, 72.1MB/s]
100% 14.2M/14.2M [00:00<00:00, 90.8MB/s]
  0% 0.00/101k [00:00<?, ?B/s]
100% 101k/101k [00:00<00:00, 104MB/s]


In [4]:
!unzip sales_train_evaluation.csv.zip
!unzip sales_train_validation.csv.zip
!unzip sample_submission.csv.zip
!unzip sell_prices.csv.zip

Archive:  sales_train_evaluation.csv.zip
  inflating: sales_train_evaluation.csv  
Archive:  sales_train_validation.csv.zip
  inflating: sales_train_validation.csv  
Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   
Archive:  sell_prices.csv.zip
  inflating: sell_prices.csv         


In [ ]:
! pip install downcast
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4
pio.orca.config.use_xvfb = True
pio.renderers.default = 'svg'

In [20]:
pio.renderers.default = 'svg'

# The below cell contains all modules and libraries required through out the program

In [1]:
# https://github.com/plotly/orca/issues/290#issuecomment-627257795

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from downcast import reduce
import plotly.express as px
from pylab import rcParams
import statsmodels as sm
from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly
import orca

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [68]:
plotly.io.orca.config.executable = '/usr/local/bin/to/'

# **Reading all CSV files required and reducing the size as the file sizes are huge**

In [2]:
# Using Pandas to reasd the files.

sales = pd.read_csv("sales_train_evaluation.csv")
sell_price = pd.read_csv("sell_prices.csv")
cal = pd.read_csv("calendar.csv")

In [ ]:
# Reading the file sizes before downcast
a = np.round(sales.memory_usage().sum()/(1024**2),1)
b = np.round(sell_price.memory_usage().sum()/(1024**2),1)
c = np.round(cal.memory_usage().sum()/(1024**2),1)

# Downcating the files. Rewriting on the same variable names reduce creating duplicates and thus saves memory usage.
sales = reduce(sales)
sell_price = reduce(sell_price)
cal = reduce(cal)

# Reading the file sizes after downcast
d = np.round(sales.memory_usage().sum()/(1024**2),1)
e = np.round(sell_price.memory_usage().sum()/(1024**2),1)
f = np.round(cal.memory_usage().sum()/(1024**2),1)

#Displaying the comparision of file size before and after downcasting.
print(f"Size of sales file before downcast was {a} MB and after downcast {d} MB", f"Size of sell_price file before downcast was {b} MB and after downcast {e} MB", \
      f"Size of calender file before downcast was {c} MB and after downcast {f} MB", sep='\n')

Size of sales file before downcast was 452.9 MB and after downcast 96.6 MB
Size of sell_price file before downcast was 208.8 MB and after downcast 45.8 MB
Size of calender file before downcast was 0.2 MB and after downcast 0.1 MB


# Sales EDA

In [ ]:
sales.head(2)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,...,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,3,0,1,1,1,3,0,1,1,0,0,0,2,0,3,5,0,0,1,1,0,2,1,2,2,1,0,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,2,1,1,0,0,0,0,0


In [3]:
print(f"Observation:\nThere are {sales.isna().sum().sum()} missing/NaN values in sales dataframe")
print(f"There are {sales['item_id'].nunique()} unique items across {sales['dept_id'].nunique()} departments under {sales['cat_id'].nunique()} categories in {sales['state_id'].nunique()} states")
days = [i for i in sales.columns.values if 'd_' in i]
print(f"There are {len(days)} days columns/features, which is around {len(days)//365} years of data")

Observation:
There are 0 missing/NaN values in sales dataframe
There are 3049 unique items across 7 departments under 3 categories in 3 states
There are 1941 days columns/features, which is around 5 years of data


In [23]:
pio.renderers

Renderers configuration
-----------------------
    Default renderer: 'svg'
    Available renderers:
        ['plotly_mimetype', 'jupyterlab', 'nteract', 'vscode',
         'notebook', 'notebook_connected', 'kaggle', 'azure', 'colab',
         'cocalc', 'databricks', 'json', 'png', 'jpeg', 'jpg', 'svg',
         'pdf', 'browser', 'firefox', 'chrome', 'chromium', 'iframe',
         'iframe_connected', 'sphinx_gallery']

In [38]:
# https://plotly.com/python/time-series/
# https://stackoverflow.com/a/27277074

date = cal.loc[:1940,'date']
temp = pd.DataFrame({'days':date, "sales":sales[days].sum(axis=0).values})
fig = px.line(temp, "days", "sales")
fig.show(renderer='notebook_connected')

 **Observation:**

 
*   Every Year on Christmas the sales greatly fall less than 50.
  
*   The demand is growing every year, not exponentially but smoothly.


In [39]:
# https://stackoverflow.com/a/60595396

temp = sales.groupby('dept_id').sum().T
date = ["".join(str(i).split('-')[:2]) for i in date]
temp.columns = temp.columns.astype(str)
temp['Date'] = date
temp = temp.groupby("Date").sum()
x_dates = [d for d in pd.date_range(start = '2011-01-29', end = '2016-04-24', freq = 'MS')]

fig = go.Figure()

for i in temp.columns:
    fig.add_trace(go.Scatter(x = x_dates, y = temp[i], mode='lines', name=str(i).title()))

fig.update_layout(title='Yearly Sales on every dept', xaxis_title='Year',yaxis_title='Total Sales', width=1000)
fig.show(renderer='notebook_connected')

In [ ]:
temp1 = sales.groupby('cat_id')[days].sum().sum(axis=1)
temp1=pd.DataFrame({'cat':temp1.index,"vals":temp1.values})
fig = px.bar(x=temp1['cat'], y=temp1['vals'], color=['red','green','blue'],width=550,height=450, labels={'x':'Categories','y':'Total Sales'})
fig.update_traces(hovertemplate='Categories: %{x} <br>Total Sales: %{y}')
fig.update_layout(showlegend=False)
fig.show()

**Observation:**
* Food sales range the highest above 40M followed by Household items around 15M and lastly Hobbies.
* Food_2 caegories are sold the most, and then household_1.
* The least sold item is Hobbies_2.
* As food is an important aspect the sales are huge on Food products.

In [ ]:
temp = sales.groupby('state_id').sum().T
date = ["".join(str(i).split('-')[:2]) for i in date]
temp.columns = temp.columns.astype(str)
temp['Date'] = date
temp = temp.groupby("Date").sum()

fig = go.Figure()

for i in temp.columns:
    fig.add_trace(go.Scatter(x = x_dates, y = temp[i], mode='lines', name=str(i).title()))

fig.update_layout(title='Monthly Sales per State', xaxis_title='Year',yaxis_title='Total Sales', width=1000, title_x=0.5)

**Observation:**
* The sales on Wisconsin and Texas are almost overlapping each other from 2013.
* Sales on California tops the chart, this could be because of the increased population on comparison to the others.
* Although the population of Wisconsin is lot less than Texas and Texas has slighlty more GDP than Wisconsin, the difference between these two states is marginal.

In [ ]:
temp = sales.groupby('store_id').sum().T
date = ["".join(str(i).split('-')[:2]) for i in date]
temp.columns = temp.columns.astype(str)
temp['Date'] = date
temp = temp.groupby("Date").sum()

fig = go.Figure()

for i in temp.columns[:4]:
    fig.add_trace(go.Scatter(x = x_dates, y = temp[i], mode='lines', name=str(i).title()))

fig.update_layout(title='Yearly Sales on California', xaxis_title='Year',yaxis_title='Total Sales', width=750, title_x=0.5)
fig.show()

**Observation:**
* Although California was leading on market sales, we can see the sales within California on different stores are very diverse.
* Store 2 in California has caught up with store 1 from June 2015. There has a peak on sales for 4 months since then and has kept the sale rate constant with respect to seasonal changes.
* Sales in store 4 has been increasing gradually every year.

In [ ]:
fig = go.Figure()

for i in temp.columns[4:7]:
    fig.add_trace(go.Scatter(x = x_dates, y = temp[i], mode='lines', name=str(i).title()))

fig.update_layout(title='Yearly Sales on Texas', xaxis_title='Year',yaxis_title='Total Sales', width=750, title_x=0.5)
fig.show()

**Observation:**
* The sales happened on Store 2 during the 2012 to 2014 was huge than the other stores, but then it has gradually falled and then tried to keep it constant on par with the other stores.

In [ ]:
fig = go.Figure()

for i in temp.columns[7:]:
    fig.add_trace(go.Scatter(x = x_dates, y = temp[i], mode='lines', name=str(i).title()))

fig.update_layout(title='Yearly Sales on Wisconsin', xaxis_title='Year',yaxis_title='Total Sales', width=750, title_x=0.5)
fig.show()

**Observation:**
* Store 3 had a great sales July 2012, and then there has been a great fall.
* That was the same time when Store 2 had a great surge in sale. This could possibly becasue there has been some internal changes on Store 2.
* Store 1 had almost overlapping sales with Store 2 before it had a surge, then store 1 had a growth in sales and had on par sales with Store 3.

In [ ]:
temp = sales.groupby('store_id').sum().T
date = ["".join(str(i).split('-')[:2]) for i in date]
temp.columns = temp.columns.astype(str)
temp['Date'] = date
temp = temp.groupby("Date").sum()

fig = go.Figure()

for i in temp.columns:
    fig.add_trace(go.Scatter(x = x_dates, y = temp[i], mode='lines', name=str(i).title()))

fig.update_layout(title='Monthly Sales per State', xaxis_title='Year',yaxis_title='Total Sales', width=1000, height=600,title_x=0.5)

**Observation:**
* We previoulsy observed that California had the most sales than Texas and Wisconsin, but itseems Store 3 in California ranks first for most sales also Store in California ranks last for the least number of sales.
* The sales on Store 1 in California differs from sales on other stores in all States by a great margin.

# **Sell Prices - EDA**

In [ ]:
sell_price = pd.read_csv("sell_prices.csv")

In [ ]:
sell_price.head(2)

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58


In [ ]:
print(f" There are {sell_price.isna().sum().sum()} missing/NaN values in sell_price dataframe")

 There are 0 missing/NaN values in sell_price dataframe


In [13]:
a = pd.date_range(start='2011-01-29', end="2016-06-19", freq='W' )
temp = sell_price.groupby('wm_yr_wk').sum().reset_index()
px.line(x = a, y = temp['sell_price'])

**Observation:**
* The total sales cost has increased every year till 2015. After 2015, the sales has almost been constant with no ups or downs.

In [ ]:
sell_price['state'] = list(map(lambda x:x[:2], sell_price['store_id']))
temp = sell_price.groupby(['wm_yr_wk','state']).sum().unstack()
temp.columns = temp.columns.droplevel()

fig = go.Figure()

for i in temp.columns:
    fig.add_trace(go.Scatter(x = a, y = temp[i], name=str(i).title()))

fig.update_layout(title='Monthly Sales per State', xaxis_title='Year',yaxis_title='Total Sales', width=800, height=500,title_x=0.5)

**Observation:**
* California tops the sales every year, whereas sales in Texas and Wisconsin are almost the same
* We can understand that same type of prodcts are being sold almost equally across all the states as the total sale price graph is sames as the number products sold on each state.

# **Calendar - EDA**

In [4]:
cal = pd.read_csv("calendar.csv")
sales = pd.read_csv("sales_train_evaluation.csv")
cal.head(2)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0


In [5]:
print(f"There are totally {cal.event_type_1.nunique()} {tuple(cal.event_type_1.unique()[1:])} types of events happening through out the year")
print(f"There are {cal.isna().sum().sum()} missing/NaN values in cal dataframe")

There are totally 4 ('Sporting', 'Cultural', 'National', 'Religious') types of events happening through out the year
There are 7542 missing/NaN values in cal dataframe


In [6]:
cal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          1969 non-null   object
 1   wm_yr_wk      1969 non-null   int64 
 2   weekday       1969 non-null   object
 3   wday          1969 non-null   int64 
 4   month         1969 non-null   int64 
 5   year          1969 non-null   int64 
 6   d             1969 non-null   object
 7   event_name_1  162 non-null    object
 8   event_type_1  162 non-null    object
 9   event_name_2  5 non-null      object
 10  event_type_2  5 non-null      object
 11  snap_CA       1969 non-null   int64 
 12  snap_TX       1969 non-null   int64 
 13  snap_WI       1969 non-null   int64 
dtypes: int64(7), object(7)
memory usage: 215.5+ KB


**Observation:**
    
> On the span of 5 years between 2011-01-29 and 2016-06-19, there were special events on 162 days out of which 5 days had multiple events.


In [15]:
cal.fillna('No_Event', inplace=True)
c = sales[days].T
d = cal[['weekday','d']][:1942]

In [16]:
temp = c.merge(d,left_index=True, right_on='d')
temp.set_index('d', inplace=True)
temp = temp.groupby('weekday').sum().sum(axis=1).sort_values()
y = [str(np.round(i/temp.sum()*100, 2))+'%' for i in temp.values]

In [17]:
colors = ['a','b','c','d','e','f','g']
fig = px.bar(x=temp.index, y=y, labels={'x':"Days","y":'Total Sales'},color=colors, text=y)
fig.update_traces(hovertemplate='Total Sales: %{y}', textposition='outside')
fig.update_layout(showlegend=False, width=800, height=500)
# fig.show()

**Observation:**
* There's a huge number of sales happening during the weekends.
* The sales rise and fall before and after weekends from Friday to Monday.

In [10]:
d = cal[['event_type_1','d']]
temp = c.merge(d,left_index=True, right_on='d')
temp.set_index('d', inplace=True)
temp = temp.groupby('event_type_1').mean().mean(axis=1).sort_values()
y = [str(np.round(i/temp.sum()*100, 2))+'%' for i in temp.values]

In [11]:
colors = ['a','b','c','d','e']
fig = px.bar(x=temp.index, y=y, labels={'x':"Days","y":'Total Sales'},color=colors, text=y)
fig.update_traces(hovertemplate='Total Sales: %{y}', textposition='outside')
fig.update_layout(showlegend=False, width=800, height=500)
fig.show()

**Observation:**
* The sales happened during sporting event times are slightly more than sales happened around no events and other events.
* The sales happened on National event days are little lower, but we can conlude it's the cause of low sales.